In [20]:
import pandas as pd 

df = pd.read_csv('./data/analyst_ratings_processed.csv', nrows=300)  # read the first 5000 (non-header) rows

# original data shape: (1400469, 4)

df.head(5)

,Unnamed: 0,title,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [21]:
import torch

In [22]:
print(torch.__version__)

2.0.1+cu117


In [23]:
print(torch.cuda.device_count())

1


In [24]:
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3060 Laptop GPU


In [25]:
print(torch.__version__)

2.0.1+cu117


In [26]:
import numpy as np

headlines_array = np.array(df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,1])

print(headlines_list)

['Agilent Reports FDA Approval For PD-L1 Companion Diagnostic On Dako Omni', 'Wells Fargo Maintains Overweight on Agilent Technologies, Raises Price Target to $95', 'Agilent Technologies Sees Q3 Adj. EPS $0.71-$0.73 vs $0.73 Est., Sales $1.225B-$1.245B vs $1.27B Est. Sees FY19 Adj. EPS $3.03-$3.07 vs $3.07 Est., Sales $5.085B-$5.125B vs $5.19B Est.', '7 Biggest Price Target Changes For Wednesday', 'Agilent Technologies Q3 Earnings Preview', 'Agilent Reports Purchase Of Luxcel Biosciences, No Terms Disclosed', "Benzinga's Top Upgrades, Downgrades For March 15, 2019", 'Agilent Companion Diagnostic Gains Expanded FDA Approval in Urothelial Carcinoma, PD-L1 IHC 22C3 pharmDx Now Approved to Identify Certain Urothelial Carcinoma Patients for Treatment With KEYTRUDA', 'Agilent Sees FY19 Adj. EPS $3-$3.05 vs $2.96 Estimate, Sales $5.13B-$5.17B vs $5.18B Est.', 'Earnings Scheduled For August 15, 2017', 'JP Morgan Healthcare Conference Continues Today, Presenters Include: Bluebird Bio, Agilent, 

In [27]:
df.shape 

(300, 4)

In [28]:
# Percentage of missing column values
null_check_percent = df.isnull().sum() * 100 / len(df)
null_check_percent

Unnamed: 0    0.0
title         0.0
date          0.0
stock         0.0
dtype: float64

In [29]:
# remove titles with missing "stock"
df = df.dropna(subset = ['stock'])
df = df.reset_index(drop = True)
null_check_percent = df.isnull().sum() * 100 / len(df)
null_check_percent

Unnamed: 0    0.0
title         0.0
date          0.0
stock         0.0
dtype: float64

In [30]:
df.shape

(300, 4)

In [31]:
df['title'] = df.title.astype(str)  # imp

In [32]:
df.head(3)

,Unnamed: 0,title,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A


In [33]:
import numpy as np

headlines_array = np.array(df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,1])

print(headlines_list)

["Shares of several healthcare companies are trading lower, selling off from this week's strength. While the US stimulus package boosted economic optimism, demand concerns still exist amid the coronavirus outbreak.", 'Wells Fargo Maintains Overweight on Agilent Technologies, Lowers Price Target to $82', 'Agilent Technologies to Acquire Advanced Analytical Technologies for $250M in Cash', "30 Stocks Moving In Wednesday's Pre-Market Session", 'A Peek Into The Markets: U.S. Stock Futures Rise; All Eyes On Fed Minutes', "Int'l. Air Transport Authority Chief Economist Sees Consolidation In Airline Sector As Some Airlines Fail, Calls Signs In Chinese Domestic Market 'Encouraging,' Load Factors Are About 60%", 'Deutsche Bank Optimistic On Fundamentals In Life Science Tools/Diagnostics Sector', 'Wells Fargo Upgrades Agilent Technologies to Outperform', 'Agilent, Nanyang Technological University Singapore to Develop Novel Approaches for Water Analysis', 'Aquinox Pharmaceuticals Announces FDA Pa

In [34]:
# %%capture

# !pip install git+https://github.com/huggingface/datasets.git
# #!pip install git+https://github.com/huggingface/transformers.git

# !pip install transformers==4.28.0

# # Monitor the training process
# !pip install wandb

In [45]:
#!huggingface-cli login # write_to_hub_token   copy here from huggingface -> settings -> access tokens -> your new write token instead of read token.

In [36]:
import torch
cuda_id = 0 # 0 for google colab and 1 for Titan XP on local computer
device = torch.device("cuda:%s" % cuda_id if torch.cuda.is_available() else "cpu")
device_name = torch.cuda.get_device_name(cuda_id) if torch.cuda.is_available() else "cpu"
print("We are using the device %s - %s" % (device, device_name))

We are using the device cuda:0 - NVIDIA GeForce RTX 3060 Laptop GPU


In [37]:
!nvidia-smi

Thu Sep 21 13:36:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.01       Driver Version: 516.01       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8    10W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [38]:
#Getting the tokenizer and the model

from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [39]:
#That's where the headlines get tokenized to be inputted into model

inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
print(inputs)

# error: TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

{'input_ids': tensor([[  101,  6661,  1997,  ...,     0,     0,     0],
        [  101,  7051, 23054,  ...,     0,     0,     0],
        [  101, 29003,  3372,  ...,     0,     0,     0],
        ...,
        [  101,  4029, 15768,  ...,     0,     0,     0],
        [  101, 15768,  2008,  ...,     0,     0,     0],
        [  101,  1037, 19043,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [40]:
#inference

outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([300, 3])


In [41]:
#Postprocessing with softmax

import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.0097, 0.9712, 0.0191],
        [0.6343, 0.2615, 0.1042],
        [0.0998, 0.0086, 0.8916],
        [0.0898, 0.1119, 0.7982],
        [0.2087, 0.0812, 0.7101],
        [0.9346, 0.0413, 0.0240],
        [0.2664, 0.0084, 0.7252],
        [0.9533, 0.0230, 0.0237],
        [0.3243, 0.0072, 0.6685],
        [0.7427, 0.0106, 0.2467],
        [0.8976, 0.0197, 0.0827],
        [0.0221, 0.0941, 0.8838],
        [0.0111, 0.8633, 0.1255],
        [0.4149, 0.3134, 0.2717],
        [0.8601, 0.0086, 0.1313],
        [0.2617, 0.0697, 0.6687],
        [0.8245, 0.0288, 0.1467],
        [0.2114, 0.0820, 0.7065],
        [0.0231, 0.1224, 0.8546],
        [0.0249, 0.6361, 0.3391],
        [0.9216, 0.0218, 0.0566],
        [0.1028, 0.0304, 0.8668],
        [0.3141, 0.0070, 0.6790],
        [0.9478, 0.0205, 0.0317],
        [0.1223, 0.0185, 0.8593],
        [0.7499, 0.0710, 0.1790],
        [0.0352, 0.0659, 0.8988],
        [0.0341, 0.0181, 0.9479],
        [0.0312, 0.0734, 0.8954],
        [0.080

In [42]:
#Model classes

model.config.id2label

{0: 'positive', 1: 'negative', 2: 'neutral'}

In [43]:
#Formatting the results as a pandas data frame

import pandas as pd

#Headline #Positive #Negative #Neutral
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

df.head(5)

,Headline,Positive,Negative,Neutral
0,Shares of several healthcare companies are tra...,0.009684,0.971235,0.019081
1,Wells Fargo Maintains Overweight on Agilent Te...,0.634309,0.261483,0.104208
2,Agilent Technologies to Acquire Advanced Analy...,0.099768,0.008599,0.891633
3,30 Stocks Moving In Wednesday's Pre-Market Ses...,0.089818,0.111947,0.798235
4,A Peek Into The Markets: U.S. Stock Futures Ri...,0.208652,0.081245,0.710103


In [44]:
#Logging the pandas table as a WandB Table
#!pip install wandb
#!wandb login YOUR_API_KEY --relogin


# all the !pip commmand are executed in anaconda terminal

import wandb

wandb.init(project="1_Sentiment_Analysis")
wandb.run.log({"Financial Sentiment Analysis Table" : wandb.Table(dataframe=df)})
wandb.run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gli-mrunal. Use `wandb login --relogin` to force relogin
